# Bank Reserve Model
A highly abstracted, simplified model of an economy, with only one type of agent and a single bank representing all banks in an economy.

People move randomly within the grid. If two or more people are on the same grid location, there is a 50% chance that they will trade with each other. If they trade, there is an equal chance of giving the other agent \\$5 or \\$2.

A positive trade balance will be deposited in the bank as savings.

If trading results in a negative balance, the agent will try to withdraw from its savings to cover the balanance. If it does not have enough savings to cover the negative balance, it will take out a loan from bank to cover the difference.

The bank is required to keep a certain percentage of deposits as reserves.

In [34]:
from mesa import Agent
from mesa import Model
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
from mesa.batchrunner import BatchRunner
from mesa.time import RandomActivation
import numpy as np
import pandas as pd
import itertools

In [35]:
class RandomWalker(Agent):
    def __init__(self,unique_id,pos,model,moore=True):
        super().__init__(unique_id,model)
        self.grid=None
        self.pos=pos
        self.x=None
        self.y=None
        self.moore=moore
    def random_move(self):
        next_moves=self.model.grid.get_neighborhood(self.pos,self.moore,True)
        next_move=self.random.choice(next_moves)
        self.model.grid.move_agent(self,next_move)

In [36]:
def cal_reserve(reserve_percent,deposits):
    return reserve_percent/100*deposits
def cal_loan(reserve,loans,deposits):
    return deposits - (reserve + loans)
class Bank(Agent):
    def __init__(self,unique_id,model,reserve_percent=50):
        super().__init__(unique_id,model)
        self.bank_loans=0
        self.reserve_percent=reserve_percent
        self.deposits=0
        self.reserves=cal_reserve(self.reserve_percent,self.deposits)
        self.bank_to_loan=0
    def bank_balance(self):
        self.reserves=cal_reserve(self.reserve_percent,self.deposits)
        self.bank_to_loan=cal_loan(self.reserves,self.bank_loans,self.deposits)        

In [37]:
class Person(RandomWalker):
    def __init__(self,unique_id,pos,model,moore,bank,rich_threshold):
        super().__init__(unique_id,pos,model,moore=moore)
        self.savings=0 #저축액
        self.loans=0 #대출
        self.wallet=self.random.randint(1,rich_threshold+1) #현재 인출된 재산
        self.wealth=0 #부
        self.customer=0 #고객은 누구?
        self.bank=bank #same bank for all
    def do_business(self):
        #돈이 있으면 혹은 은행이 사람들에게 빌려줄 돈이 있으면
        if self.savings>0 or self.wallet>0 or self.bank.bank_to_loan>0:
            my_cell=self.model.grid.get_cell_list_contents([self.pos,]) #거래 가능한 사람들
            if len(my_cell) > 1: #셀에 무엇인가 있다고 할때,
                customer=self
                while True: #do-while
                    while True: #do-while
                        customer=self.random.choice(my_cell) #아무나 하나
                        if isinstance(customer,Person): #Person인가?
                            break
                    if customer != self: #본인이 아닌가?
                        break
                if self.random.randint(0,1)==0: #50% 확률로 거래 성사
                    trading_amount = 5 if self.random.randint(0,1)==0 else 2 #거래 규모 5? 2?
                    customer.wallet+=trading_amount #거래(매입)
                    self.wallet-=trading_amount
    def balance_books(self):
        if self.wallet<0: #거래에서 외상 매입했다면
            if self.savings>=(self.wallet*-1): #저축으로 충분하다면,
                self.withdraw_from_savings(self.wallet*-1) #인출함
            else:
                if self.savings>0: #잔고가 있다면 잔고 전부를 인출
                    self.withdraw_from_savings(self.savings)
                temp_loan=self.bank.bank_to_loan #은행에서 대출 여력
                if temp_loan>=(self.wallet*-1): #대출여력이 은행에 있으면
                    self.take_out_loan(self.wallet*-1) #대출
                else:
                    self.take_out_loan(temp_loan) #모두 대출
        else: #외상 매입이 아니라면
            self.deposit_to_savings(self.wallet) #전부를 은행에 저축
        if self.loans>0 and self.savings>0: #대출이 있고 저축도 있으면
            if self.savings>=self.loans: #저축액이 대출보다 많음
                self.withdraw_from_savings(self.loans) #대출만큼 인출
                self.repay_a_loan(self.loans) #대출 값음
            else:
                self.withdraw_from_savings(self.savings) #저축만큼 인출
                self.repay_a_loan(self.wallet) #지갑에 있는 것 갚음
        self.wealth=self.savings-self.loans #총계정
    def deposit_to_savings(self,amount):
        self.wallet-=amount
        self.savings+=amount
        self.bank.deposits+=amount
    def withdraw_from_savings(self,amount):
        self.wallet+=amount
        self.savings-=amount
        self.bank.deposits-=amount
    def repay_a_loan(self,amount):
        self.loans-=amount
        self.wallet-=amount
        self.bank.bank_to_loan+=amount
        self.bank.bank_loans-=amount
    def take_out_loan(self,amount):
        self.loans+=amount
        self.wallet+=amount
        self.bank.bank_to_loan-=amount
        self.bank.bank_loans+=amount
    def step(self):
        self.random_move()
        self.do_business()
        self.balance_books()
        self.bank.bank_balance()

In [38]:
def get_num_rich_agents(m): #m=model
    rich_agents=[a for a in m.schedule.agents if a.savings>m.rich_threshold]
    return len(rich_agents)
def get_num_poor_agents(m):
    poor_agents=[a for a in m.schedule.agents if a.loans>10]
    return len(poor_agents)
def get_num_mid_agents(m):
    mid_agents=[a for a in m.schedule.agents if a.loans<10 and a.savings<m.rich_threshold]
    return len(mid_agents)
def get_total_savings(m):
    agent_savings=[a.savings for a in m.schedule.agents]
    return np.sum(agent_savings)
def get_total_wallets(m):
    agent_wallets=[a.wallet for a in m.schedule.agents]
    return np.sum(agent_wallets)
def get_total_money(m):
    wallet_money=get_total_wallet(m)
    savings_money=get_total_savings(m)
    return wallet_money+savings_money
def get_total_loans(m):
    agent_loans=[a.loans for a in m.schedule.agents]
    return np.sum(agent_loans)
def track_params(m):
    return (m.init_people,m.rich_threshold,m.reserve_percent)
def track_run(m):
    return m.uid
class BankReserves(Model):
    id_gen=itertools.count(1)
    def __init__(self,height=20,width=20,init_people=2,rich_threshold=10,reserve_percent=50):
        self.uid=next(self.id_gen)
        self.height=height
        self.width=width
        self.init_people=init_people
        self.schedule=RandomActivation(self)
        self.grid=MultiGrid(self.width,self.height,torus=True)
        self.rich_threshold=rich_threshold
        self.reserve_percent=reserve_percent
        self.datacollector=DataCollector(
            model_reporters={
                "Rich":get_num_rich_agents,
                "Poor":get_num_poor_agents,
                "MiddleClass":get_num_mid_agents,
                "Savings":get_total_savings,
                "Wallets":get_total_wallets,
                "Money":get_total_money,
                "Loans":get_total_loans,
                "ModelParams":track_params,
                "Run":track_run,
            },
            agent_reporters={"Wealth":lambda x : x.wealth},
        )
        self.bank=Bank(1,self,self.reserve_percent)
        for i in range(self.init_people):
            x=self.random.randrange(self.width)
            y=self.random.randrange(self.height)
            p=Person(i,(x,y,),self,True,self.bank,self.rich_threshold)
            self.grid.place_agent(p,(x,y,))
            self.schedule.add(p)
        self.running=True
        self.datacollector.collect(self)
    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)

In [39]:
br_params={
    "init_people":[25,100,150,200],
    "rich_threshold":[5,10,15,20],
    "reserve_percent":[0,50,100]
}
br=BatchRunner(
    BankReserves,
    br_params,
    iterations=1,
    max_steps=100,
    model_reporters={"DataCollector":lambda m:m.datacollector}
)
br.run_all()
br_df=br.get_model_vars_dataframe()
br_step_data=pd.DataFrame()
for i in range(len(br_df['DataCollector'])):
    if isinstance(br_df['DataCollector'][i],DataCollector):
        i_run_data=br_df['DataCollector'][i].get_model_vars_dataframe()
        br_step_data=br_step_data.append(i_run_data,ignore_index=True)
br_step_data.to_csv("BankReserveModel_Step_Data.csv")

48it [00:14,  1.86it/s]
